**European Soccer Database Predictions**

Imports :

In [1]:
import sqlite3
import pandas as pd
import math
from datetime import datetime
from array import *
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

Première étape : Fetch

On récupère les différentes tables nécéssaires dans la DB et on les fusionne pour ne faire qu'une seule table. Nous avons bien sûr pris la table des matchs, c'est la base même de la prédiction. Nous avons ensuite ajouté la table player attributes, qui regroupe les différentes statistiques utiles à propos des joueurs présents sur le terrain. En effet, la présence d'un bon joueur influe de façon évidente sur le résultat du match. Pour cela pour pas compliquer trop les choses nous avons choisi de prendre que la cote de potentiel des joueurs ainsi que leur overall rating.

In [2]:
def queryBuilder(lettre, df, team, table):
    for element in range(1,12):
        queryAExecuter = "SELECT x.id,x.date,"
        queryAExecuter = queryAExecuter +lettre+str(element)+".potential, "+ lettre+str(element)+".overall_rating,MAX("+lettre+str(element)+".date)"
        queryAExecuter = queryAExecuter + " FROM " + table + " x "                
        queryAExecuter = queryAExecuter + " LEFT JOIN Player_Attributes "+lettre+str(element)+" ON "+lettre+str(element)+".player_api_id = x."+team+"_player_"+str(element)+" AND "+ lettre +str(element)+".date <= x.date"
        queryAExecuter = queryAExecuter + " GROUP BY x.id ORDER BY x.date "
        queryEnCours = dat.execute(queryAExecuter)
        colsEnCours = [column[0] for column in queryEnCours.description]
        ncols = []
        for i, name in enumerate(colsEnCours):
            if i!=0 and i!=1:
                ncols.append(name + "_" + team + str(element))
                colsEnCours[i] = name + "_" + team + str(element)
            else:
                ncols.append(name)
        dataFrameEnCours = pd.DataFrame.from_records(data = queryEnCours.fetchall(), columns = ncols)
        dataFrameEnCours.fillna(-50)
        dataFrameEnCours = dataFrameEnCours.drop(columns = ['id','date'])
        for i, name in enumerate(colsEnCours):
            if name != 'id' and name != 'date':
                df.insert(2, name, dataFrameEnCours[name], True) 
        del dataFrameEnCours

Nous avons donc créé une fonction queryBuilder avec quatres paramètres afin de ne pas avoir du code repetitif pour la création des query pour les caractéristiques des joueurs, qu'ils soient chez eux ou en déplacements. Cela permet aussi de faire exactement les mêmes modifications, que ce soit sur la table de Test ou la table de Train.

Nous n'avons pas rajouté la table Joueur car nous avons remarqué qu'au final tous les joueurs sont des sportifs du coup le poids des joueurs ainsi que leur taille n'influence en rien l'équipe , leur age non plus.

In [3]:
dat = sqlite3.connect('/kaggle/input/jvm000001/database/database.sqlite')

table = ["Country", "League", "X_Train", "Player", "Player_Attributes", 
         "Team", "Team_Attributes"]
queryTeamAttributes = dat.execute("SELECT * FROM Team_Attributes")
queryTrain = "SELECT x.* FROM X_train x"
queryXTrain = dat.execute(queryTrain) 
colsXTrain = [column[0] for column in queryXTrain.description]
X_train = pd.DataFrame.from_records(data = queryXTrain.fetchall(), columns = colsXTrain)
queryXTest = dat.execute("SELECT * FROM X_test") 
colsXTest = [column[0] for column in queryXTest.description]
X_test = pd.DataFrame.from_records(data = queryXTest.fetchall(), columns = colsXTest)
queryBuilder('a',X_train,'home',"X_train")
queryBuilder('b',X_train,'away',"X_train")
queryBuilder('a',X_test,'home',"X_test")
queryBuilder('b',X_test,'away',"X_test")

In [4]:
print(X_train.describe)

<bound method NDFrame.describe of        index     id MAX(b11.date)_away11  overall_rating_away11  \
0          0  11103                 None                    NaN   
1          1  16167                 None                    NaN   
2          2  11222                 None                    NaN   
3          3  11018                 None                    NaN   
4          4  25596                 None                    NaN   
...      ...    ...                  ...                    ...   
20474  20474   3902  2016-05-19 00:00:00                   74.0   
20475  20475  13531  2015-10-30 00:00:00                   67.0   
20476  20476  15802  2016-02-04 00:00:00                   74.0   
20477  20477  23728  2015-09-21 00:00:00                   74.0   
20478  20478   3255  2016-03-03 00:00:00                   54.0   

       potential_away11 MAX(b10.date)_away10  overall_rating_away10  \
0                   NaN                 None                    NaN   
1                  

Deuxième étape : Clean

Nous retirons les différentes colonnes qui ne concernent pas directement le jeu lui même. Comme par exemple : la ville d'origine de l'équipe etc, sont des facteurs qui n'influencent que très peu les résultats, voir pas dutout. Il est important de se focaliser d'abord sur les informations les plus importantes.

Dans les attributs d'équipes, nous avons aussi retiré les class car nous pensons que des chiffres sont bien plus précis que des noms de class.

Nous n'avons pas supprimé dans X_train les colonnes de joueur avec leur position X et Y car nous pensons qu'il est interessant de savoir la position de la personne car certaines d'entre elles sont plus forte dans une position qu'une autre.

Nous préparons les informations afin qu'elles soient plus faciles à utiliser. Au niveau du prepare certaines préparations sont aussi faites dans le queryBuilder car on souhaitait d'abord nettoyer et préparer les informations avant de les rajouter dans notre X_train.

In [5]:
X_train = X_train.drop(columns=['season','date','league_id','country_id'])
for x in range(1,12):
    X_train = X_train.drop(columns=["MAX(a"+str(x)+".date)_home"+str(x)])
    X_train = X_train.drop(columns=["MAX(b"+str(x)+".date)_away"+str(x)])
for x in range(1,12):
    X_test = X_test.drop(columns=["MAX(a"+str(x)+".date)_home"+str(x)])
    X_test = X_test.drop(columns=["MAX(b"+str(x)+".date)_away"+str(x)])
X_test = X_test.drop(columns=['season','date','league_id','country_id'])

Troisième étape : Prepare

Nous préparons les informations afin qu'elles soient plus faciles à utiliser. Au niveau du prepare certaines préparations sont aussi faites dans le queryBuilder car nous souhaitions d'abord nettoyer et préparer les informations avant de les rajouter dans notre X_train.

In [6]:
X_train = X_train.fillna(-9999)
X_test = X_test.fillna(-9999)
labels_catA = X_train["home_team_goal"]
labels_catB = X_train["away_team_goal"]
labels_num = [None]* len(X_train.index)
for index,rows in X_train.iterrows():
    y= rows["home_team_goal"]
    z= rows["away_team_goal"]
    if y > z:
        labels_num[index] = 1
    elif y < z:
        labels_num[index] = -1
    else :
        labels_num[index] = 0

X_train = X_train.drop(columns=['home_team_goal','away_team_goal']);

Voici le contenu au niveau des colonnes de notre table X_train.

In [7]:
print(X_train.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20479 entries, 0 to 20478
Data columns (total 116 columns):
index                    int64
id                       int64
overall_rating_away11    float64
potential_away11         float64
overall_rating_away10    float64
potential_away10         float64
overall_rating_away9     float64
potential_away9          float64
overall_rating_away8     float64
potential_away8          float64
overall_rating_away7     float64
potential_away7          float64
overall_rating_away6     float64
potential_away6          float64
overall_rating_away5     float64
potential_away5          float64
overall_rating_away4     float64
potential_away4          float64
overall_rating_away3     float64
potential_away3          float64
overall_rating_away2     float64
potential_away2          float64
overall_rating_away1     float64
potential_away1          float64
overall_rating_home11    float64
potential_home11         float64
overall_rating_home10    float64
pot

In [8]:
print(X_train.describe)

<bound method NDFrame.describe of        index     id  overall_rating_away11  potential_away11  \
0          0  11103                -9999.0           -9999.0   
1          1  16167                -9999.0           -9999.0   
2          2  11222                -9999.0           -9999.0   
3          3  11018                -9999.0           -9999.0   
4          4  25596                -9999.0           -9999.0   
...      ...    ...                    ...               ...   
20474  20474   3902                   74.0              80.0   
20475  20475  13531                   67.0              67.0   
20476  20476  15802                   74.0              74.0   
20477  20477  23728                   74.0              74.0   
20478  20478   3255                   54.0              61.0   

       overall_rating_away10  potential_away10  overall_rating_away9  \
0                    -9999.0           -9999.0               -9999.0   
1                    -9999.0           -9999.0       

On separe notre Dataframe de train et les prédictions déjà connu en train et test.

In [9]:
features_train, features_test, labels_train, labels_test = train_test_split(X_train, labels_num)

Nous avons commencé par un Modèle Logistic Regression car il nous semblait être le plus simple à mettre en place afin de pouvoir tester le plus facilement.
Malheureusement nous avons remarqué que même avec de l'optimisation de paramètre avec un gridSearch ou ne depassait pas les 47%.

In [10]:
log_reg = SGDClassifier(loss='log', max_iter=5, tol=-np.infty, random_state=42)
#log_reg = SGDClassifier(alpha= 0.001,fit_intercept= False,l1_ratio= 1, loss='log', max_iter=5, n_jobs= -1, penalty= 'l1', random_state= 42, tol=-np.infty)
log_reg.fit(features_train, labels_train)
labels_pred = log_reg.predict(features_test)
accuracy_log = accuracy_score(labels_test, labels_pred)
print ("Accuracy logistic regression= ", accuracy_log)

#ici on fait la prediction avec 
labels_pred = log_reg.predict(X_test)

Accuracy logistic regression=  0.351171875


C'est pour cela que nous avons changé et nous sommes parti vers du KNN. En y réfléchissant, nous nous sommes dit que vu que la sortie de notre modèle est soit 0,-1 ou 1, le KNN peut etre une bonne idée vu que celui-ci crée des groupes donc nous avons décidé de l'implémenter. Au début du code ci-dessous, nous avons fait une boucle for qui permet de rechercher quel est le meilleur nombre de voisins K afin de savoir avec combien de valeurs notre modèle va la comparer avant de faire sa prédiction. Nous avons d'abord commencé par 40 puis après nous avons augmenté petit à petit car nous avions remarqué que lorsque nous mettions de 1 à 40 non compris il prenait 39. Nous nous sommes alors dit qu'il y avait surement moyen d'avoir un K plus haut qui serait mieux pour notre modèle. Nous avons ensuite testé 100 puis 200 et nous avons fini avec 300. Lorsque nous avons testé le 300, le k le plus haut était 253. Nous pensions que nous avions atteint le maximum. Et nous l'avons verifié dans le GridSearchCV qui est un peu plus bas et c'est bien le 253 qui est le meilleur. Certes c'est une grosse valeur mais cela permet d'être plus précis et niveau des performances et du temps cela ne prend pas spécialement beaucoup plus de temps. Nous sommes donc arrivé à une accuracy de 0.482482.

La partie de code qui suit prend pas mal de temps.
La console affiche donc ceci :


nbDeVoisins = [i for i in range(1, 300, 2)]
scores = []
from sklearn.model_selection import cross_val_score

for k in nbDeVoisins:
    knn = KNeighborsClassifier(n_neighbors=k,weights='uniform',metric='manhattan')
    score = cross_val_score(knn, features_train, labels_train, cv=5)
    scores.append(score.mean())
    print(k)

k_optimal_pour_knn = nbDeVoisins[scores.index(max(scores))]
print("Optimal k = " + k_optimal_pour_knn)

Ici on a donc le GridSearch qui va nous permettre de trouver les meilleurs paramètres pour notre Modèle de Logistic Regression

In [11]:
parametres = [{'loss':['log'], 'max_iter':[5], 'tol': [-np.infty], 'random_state':[42]},{'penalty' : ['none','l1', 'l2', 'elasticnet'],'alpha': [0.001],
    'l1_ratio': [1],'loss': ['log'],'fit_intercept' : [False,True], 'max_iter' : [5],'tol' : [-np.infty], 'n_jobs' : [-1], 'random_state' : [42]}]
grid_search = GridSearchCV(estimator = log_reg,
                          param_grid = parametres,
                          scoring = 'accuracy',
                          cv = 10,
                          n_jobs = -1)
grid_search = grid_search.fit(features_train, labels_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.3713132365388372
{'alpha': 0.001, 'fit_intercept': False, 'l1_ratio': 1, 'loss': 'log', 'max_iter': 5, 'n_jobs': -1, 'penalty': 'l1', 'random_state': 42, 'tol': -inf}


Ici on a donc le code qui contient notre modèle avec justement nos 253 voisins.

In [12]:
optimal_knn = KNeighborsClassifier(n_neighbors=253,weights= 'uniform',metric= 'euclidean')
# on remarque qu'avec ce cross validation on arrive à un tableau de score qui se rapporche 
#les un des autres donc cela montre bien que l'on est proche et que l'on a pas d'overfitting du modèle
optimal_knn.fit(features_train, labels_train)
labels_pred = optimal_knn.predict(features_test)

accuracy_KNN = accuracy_score(labels_test, labels_pred)

print ("Accuracy du KNN = ", accuracy_KNN)

labels_pred = optimal_knn.predict(X_test)

Accuracy du KNN =  0.4462890625


Notre code pour la gridSearch pour les paramètres optimaux pour notre modèle KNN

In [13]:
parametres = [{'n_neighbors' : [3,5,11,19,253],'weights' : ['uniform','distance'], 'metric':['euclidean','manhattan']}]
grid_search = GridSearchCV(estimator = optimal_knn,
                          param_grid = parametres,
                          scoring = 'accuracy',
                          cv = 10,
                          n_jobs = -1)
grid_search = grid_search.fit(features_train, labels_train)
print(grid_search.best_score_)
print(grid_search.best_params_)

0.46168370336610454
{'metric': 'euclidean', 'n_neighbors': 253, 'weights': 'uniform'}


Les parties de code ci-dessous nous permettent de mettre notre id de match ainsi que notre prédiction dans un DataFrame et ensuite on le convertit en un fichier CSV.

In [14]:
dataframeToSend = pd.DataFrame()
dataframeToSend['id'] = X_test['index']
dataframeToSend['classes'] = pd.DataFrame(labels_pred)

import os
def export_to_csv(df):
    #os.remove("dataframe.csv")
    df.to_csv('dataframe.csv',index=False)
    
export_to_csv(dataframeToSend)